In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from pandas import ExcelWriter

import math
import seaborn as sns
from matplotlib.pyplot import figure
from sklearn.metrics import f1_score, accuracy_score
import os
from keras import regularizers
from numpy import array, hstack, vstack
from sklearn.utils import shuffle

from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from matplotlib import pyplot

# !pip install tsfresh --user
import tsfresh

from tsfresh.examples.har_dataset import download_har_dataset, load_har_dataset, load_har_classes
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import settings

# для построения моделей воспользуемся sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


os.chdir("C:/Users/VIP13/Статья 2/Котировки 6")

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols = list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# put it all together
	agg = concat(cols, axis=1)
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg.values

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test, :], data[-n_test:, :]
 
# fit an xgboost model and make a one step prediction
def xgboost_forecast(train,eta, gamma, max_depth):
	# transform list into array
	train = asarray(train)
	# split into input and output columns
	trainX, trainy = train[:, :-1], train[:, -1]
	# fit model
	model = XGBRegressor(objective='reg:squarederror', n_estimators=100, eta=eta, gamma=gamma, max_depth=max_depth)
	model.fit(trainX, trainy)
	# make a one-step prediction
# 	yhat = model.predict(asarray([testX]))
	return model
 

In [ ]:
frame_3=pd.read_csv('all_snp_btc_rv_rr.csv', delimiter=';')
frame=frame_3.iloc[:,1:].copy()
frame=frame.loc[:, [c for c in frame.columns if (frame[c]>0).all()]]
frame=np.log(np.sqrt(frame))
ind=[i for i in range(0,len(frame))]
frame.index=ind
frame.head()
data=frame['rv_shk_btc']
# data=frame['rv_shk_snp']

In [ ]:
frame_learn=pd.DataFrame(datas)
frame_transf=features_filtered_direct.iloc[:,1:]

In [ ]:

n_in=5
len_wind=399
step_smooth=5

iteratation_5=int((len(data)-len_wind)/step_smooth)+1

tmp_prog=[]
tmp_orig=[]
for i in range(0, iteratation_5):
    datas=data.iloc[0+step_smooth*i:len_wind+step_smooth*i]

    X=pd.DataFrame(datas.iloc[:-1])
    y=datas.iloc[1:]
    y.index=np.arange(0,len(y),1)
    X.index=y.index
    X['id']=y.index
        
    features_filtered_direct = extract_relevant_features(X,y, column_id='id')

    frame_learn=pd.DataFrame(datas)
    frame_transf=features_filtered_direct.iloc[:,1:]

    columnss=[]
    columnss.append(frame_learn.columns[0])
    for k in range(0,len(frame_transf.columns)):
        columnss.append(str(frame_transf.columns[k]))

    prog=pd.DataFrame(columns=columnss)
    orig=pd.DataFrame(columns=columnss)



    frame_learn_use=pd.DataFrame(frame_learn)
    frame_learn_use=series_to_supervised(frame_learn_use, n_in=n_in)

    for j in range(0,len(frame_transf.columns)):
        fr2=pd.DataFrame(frame_transf[frame_transf.columns[j]])
        fr2=series_to_supervised(fr2, n_in=n_in)
        frame_learn_use=vstack([frame_learn_use, fr2])

    model = xgboost_forecast(frame_learn_use, eta=0.2, gamma=0.3, max_depth=8)



    series=DataFrame(data.iloc[0+step_smooth*i:len_wind+1+step_smooth*i])
    values = series.values
    data_test = series_to_supervised(values, n_in=n_in)[-1]
    testX, testy = data_test[:-1], data_test[-1]
    yhat = model.predict(asarray([testX]))[0]
    tmp_prog.append(yhat)
    tmp_orig.append(testy)
    print(i)


In [ ]:
abs((np.exp(tmp_orig)-np.exp(tmp_prog))/np.exp(tmp_orig)).mean()

In [ ]:
tmp_prog=np.exp(tmp_prog)

pd.DataFrame(tmp_prog).to_csv('tsfresh_btc.csv')
# pd.DataFrame(tmp_prog).to_csv('tsfresh_snp.csv')